### Load wav

In [4]:
import librosa
import numpy as np
import tensorflow as tf

In [5]:
X, sample_rate = librosa.load('female_angry.wav', sr=48000, duration=4)

In [6]:
X.shape

(131072,)

In [7]:
t = tf.convert_to_tensor(X, dtype=tf.float32)

In [8]:
t

<tf.Tensor 'Const:0' shape=(131072,) dtype=float32>

In [21]:
type(X[0])

numpy.float32

In [22]:
librosa.output.write_wav('my_audiof.wav', X, sr=48000)

In [23]:
speech_bytes = X.tostring()

In [24]:
len(speech_bytes)

524288

In [27]:
rec_X = np.fromstring(speech_bytes, dtype=np.uint8).astype(np.float32)
rec_X.shape

/home/datahack/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if __name__ == '__main__':


(524288,)

In [5]:
type(sample_rate)

int

In [6]:
sample_rate

48000

In [ ]:
import numpy as np

In [ ]:
sample_rate = np.array(sample_rate)

In [ ]:
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)

In [ ]:
feature = mfccs

In [ ]:
feature.shape

### Load raw

In [ ]:
import sox

In [ ]:
tfm = sox.Transformer()

In [ ]:
tfm.set_input_format(file_type='raw', rate=48000, bits=16, channels=4, encoding='signed-integer')

In [ ]:
tfm.set_output_format(file_type='wav', rate=48000, bits=16, channels=4, encoding='signed-integer')

In [ ]:
output_tuple = tfm.build(input_filepath='raw_1551089459.26.raw', output_filepath=None, return_output=True)

In [ ]:
output_tuple[2]

### Use pysoundfile to read RAW and building our own WAV

In [ ]:
import soundfile as sf

In [ ]:
data, rate = sf.read('raw_1551260691.73.raw', channels=4, samplerate=48000, subtype='PCM_16', endian='LITTLE')

In [ ]:
data.shape

In [ ]:
data

In [ ]:
import math
import struct

#def to_bytes(n, length, endianess='big'):
#    h = '%x' % n
#    s = ('0'*(len(h) % 2) + h).zfill(length*2).decode('hex')
#    return s if endianess == 'big' else s[::-1]

# Generates the .wav file header for a given set of samples and specs
def genHeader(sampleRate, bitsPerSample, channels, samples):
    datasize = len(samples) * channels * bitsPerSample // 8
    o = bytes("RIFF").encode('ascii')                                      # (4byte) Marks file as RIFF
    #o += to_bytes(datasize + 36, 4,'little')                              # (4byte) File size in bytes excluding this and RIFF marker
    o += struct.pack('<I',datasize + 36)                                   # (4byte) File size in bytes excluding this and RIFF marker
    o += bytes("WAVE").encode('ascii')
    o += bytes("fmt ").encode('ascii')                                     # (4byte) Format Chunk Marker
    #o += to_bytes(16, 4,'little')                                         # (4byte) Length of above format data
    o += struct.pack('<I',16)                                              # (4byte) Length of above format data                     
    #o += to_bytes(1, 2,'little')                                          # (2byte) Format type (1 - PCM)
    o += struct.pack('<H',1)                                               # (2byte) Format type (1 - PCM)
    #o += to_bytes(channels, 2,'little')                                   # (2byte)
    o += struct.pack('<H',channels)                                        # (2byte)
    #o += to_bytes(sampleRate, 4,'little')                                 # (4byte)
    o += struct.pack('<I', sampleRate)                                     # (4byte)
    #o += to_bytes((sampleRate * channels * bitsPerSample) // 8, 4,'little')# (4byte)
    o += struct.pack('<I',(sampleRate * channels * bitsPerSample) // 8)    # (4byte)
    #o += to_bytes((channels * bitsPerSample) // 8, 2,'little')            # (2byte)
    o += struct.pack('<H', (channels * bitsPerSample) // 8)                # (2byte)
    #o += to_bytes(bitsPerSample, 2,'little')                              # (2byte)
    o += struct.pack('<H', bitsPerSample)                                  # (2byte)
    o += bytes("data").encode('ascii')                                     # (4byte) Data Chunk Marker
    #o += to_bytes(datasize, 4,'little')                                    # (4byte) Data size in bytes
    o += struct.pack('<I',datasize)                                    # (4byte) Data size in bytes
    return o

# Expects samples in the form [[channel1sample0,channel2sample0,...],[channel1sample1,channel2sample1,...]]
# Samples are values between -1 and 1 then upscaled to the correct amplitude (-2**15 -> 2**15 roughly) for 16 bit
def genFile(sampleRate, bitsPerSample, channels, samples):
    with open("test.wav","wb") as f:
        f.write(genHeader(sampleRate, bitsPerSample, channels, samples))
        m = 2**(bitsPerSample-1)-1
        channels = range(channels)
        for sample in samples:
            for c in channels:                
                #f.write(struct.pack('<h', int(sample[c] * m)))
                print(str(type(struct.pack('<h', int(sample[c] * m)))))
                #print(struct.pack('<h', int(sample[c] * m)))

genFile(48000, 16, 4, data)

### Getting MFCC from WAV

In [ ]:
import librosa
import numpy as np
import pandas as pd
import struct

In [58]:
signal = np.load('/home/datahack/catkin_ws/src/mic2wav/src/wav_files/audio.npy')

In [65]:
signal.shape

(250750,)

In [64]:
genWave(signal.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/AUDIO_WAV.wav', sampleRate=20000)

1
20000


In [ ]:
gen

In [ ]:
signal, sample_rate = librosa.core.load('/home/datahack/catkin_ws/src/mic2wav/src/wav_files/raw_CLEAN.wav', sr=48000)

In [ ]:
len(signal.shape)

In [ ]:
def transform_signal(signal, sr_ori=48000, sr_des=44100, mono=True, fix_length=110250):
    signal_shape = signal.shape
    nRows = signal_shape[0]
    nCols = signal_shape[1]
    if mono & (len(signal_shape) == 2):
        if nRows<nCols:
            signal_T = signal.T
            signal = signal_T.mean(axis=1)
        else:
            signal = signal.mean(axis=1)
            
    signal_res = librosa.resample(signal, orig_sr=sr_ori, target_sr=sr_des, res_type='kaiser_best')
    
    if mono & (fix_length is not None):
        signal_res_fix = librosa.util.fix_length(signal_res, size=fix_length)
        return signal_res_fix
    else:
        return signal_res

In [ ]:
signal_mono = signal.mean(axis=1)

In [ ]:
signal_mono_res = librosa.resample(signal_mono, orig_sr=48000, target_sr=44100, res_type='kaiser_fast')

In [ ]:
signal_mono_res.shape

In [ ]:
sample_rate = np.array(44100)

In [ ]:
signal_fix = librosa.util.fix_length(signal_mono_res, size=110250)

In [ ]:
librosa.get_duration(signal_fix, sr=44100)

In [ ]:
mfcc_fix = librosa.feature.mfcc(signal_fix, sr=44100, n_mfcc=13)

In [ ]:
mfcc_fix.shape

In [ ]:
mfcc_input_test = mfcc_fix.mean(axis=0)

In [ ]:
mfcc_input = mfcc_input_test.reshape(-1,1)

In [ ]:
mfcc_input_test = np.expand_dims(mfcc_input, axis=0)

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model("Emotion_Voice_Detection_Model.h5")

In [ ]:
preds = model.predict(mfcc_input_test)

In [ ]:
preds.argmax(axis=1)

In [ ]:
preds

### Transform WAV

In [59]:
import numpy as np
import struct

In [60]:
def genHeader(samples, sampleRate, bitsPerSample, channels):
    datasize = len(samples) * channels * bitsPerSample // 8    
    o = bytes("RIFF").encode('ascii')                                      # (4byte) Marks file as RIFF
    o += struct.pack('<I',datasize + 36)                                   # (4byte) File size in bytes excluding this and RIFF marker
    o += bytes("WAVE").encode('ascii')
    o += bytes("fmt ").encode('ascii')                                     # (4byte) Format Chunk Marker
    o += struct.pack('<I',16)                                              # (4byte) Length of above format data                     
    o += struct.pack('<H',1)                                               # (2byte) Format type (1 - PCM)
    o += struct.pack('<H',channels)                                        # (2byte)
    o += struct.pack('<I', sampleRate)                                     # (4byte)
    o += struct.pack('<I',(sampleRate * channels * bitsPerSample) // 8)    # (4byte)
    o += struct.pack('<H', (channels * bitsPerSample) // 8)                # (2byte)
    o += struct.pack('<H', bitsPerSample)                                  # (2byte)
    o += bytes("data").encode('ascii')                                     # (4byte) Data Chunk Marker
    o += struct.pack('<I',datasize)                                        # (4byte) Data size in bytes

    return o

In [61]:
def genWave(samples, filename, sampleRate=44100, bitsPerSample=16, channels=1):
    header = genHeader(samples, sampleRate, bitsPerSample, channels)
    print(channels)
    print(sampleRate)
    body = ""
    channels = np.arange(channels)
    for sample in samples:                
        for c in channels:     
            sf = sample[c]
            #sf = float(s) / float(32768)
            #if(sf>1):
            #    sf = 1
            #if(sf<-1):
            #    sf = -1

            body += struct.pack('<h', int(sf*32767))
    
    f = open(filename, 'wb')
    f.write(header + body)
    f.close()

### Prueba servir

In [1]:
import requests

In [49]:
response = requests.get('http://192.168.1.21:9000/synthesize', params={'text': 'Me gusta ir de putas ocasionalmente'})

In [50]:
f = open('test6.wav', 'wb')
f.write(response.content)
f.close()

In [51]:
response.content

'RIFF\xa48\x01\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00 N\x00\x00@\x9c\x00\x00\x02\x00\x10\x00data\x808\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

### Noise

#### Visualization

In [ ]:
import librosa.display
import matplotlib.pyplot as plt

In [ ]:
test1,sr1 = librosa.core.load('/home/datahack/catkin_ws/src/mic2wav/src/wav_files/male_fearful.wav', sr=48000, mono=False)

In [ ]:
test_transformed1 = transform_signal(test1)

In [ ]:
% matplotlib inline


plt.figure(figsize=(12, 4))
librosa.display.waveplot(test_transformed1, sr=44100)

In [ ]:
mfcc_test1 = np.mean(librosa.feature.mfcc(test_transformed1, sr=44100, n_mfcc=13), axis=0)

In [ ]:
mfcc_test1_input = np.expand_dims(mfcc_test1.reshape(-1,1), axis=0)

In [ ]:
test2,sr2 = librosa.core.load('/home/datahack/catkin_ws/src/mic2wav/src/wav_files/female_sad.wav', sr=48000, mono=False)

In [ ]:
test_transformed2 = transform_signal(test2)

In [ ]:
% matplotlib inline


plt.figure(figsize=(12, 4))
librosa.display.waveplot(test_transformed2, sr=44100)

In [ ]:
mfcc_test2 = np.mean(librosa.feature.mfcc(test_transformed2, sr=44100, n_mfcc=13), axis=0)

In [ ]:
mfcc_test2_input = np.expand_dims(mfcc_test2.reshape(-1,1), axis=0)

In [ ]:
test_wav = librosa.resample(test, orig_sr=48000, target_sr=44100)

In [ ]:
test_fixed = librosa.util.fix_length(test_wav, size=110250)

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.waveplot(test_fixed, sr=44100)

In [ ]:
noise,sr_noise = librosa.core.load('/home/datahack/catkin_ws/src/mic2wav/src/wav_files/noise2.wav', sr=48000)

In [ ]:
noise_wav = librosa.resample(noise, orig_sr=48000, target_sr=44100)

In [ ]:
noise_fixed = librosa.util.fix_length(noise_wav, size=110250)

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.waveplot(noise_fixed, sr=44100)

In [ ]:
np.histogram(noise_fixed)

#### Reduce Noise

In [ ]:
from pysndfx import AudioEffectsChain
import math
import python_speech_features
import scipy as sp

In [ ]:
genWave(test1.T, '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1.wav', sampleRate=48000, channels=4)

In [ ]:
test1Transformed = transform_signal(test1, fix_length=None)

In [ ]:
genWave(test1Transformed.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1Transformed.wav', sampleRate=44100, channels=1)

In [ ]:
'''------------------------------------
NOISE REDUCTION USING POWER:
    receives an audio matrix,
    returns the matrix after gain reduction on noise
------------------------------------'''
def reduce_noise_power(y, sr):

    cent = librosa.feature.spectral_centroid(y=y, sr=sr)

    threshold_h = round(np.median(cent))*1.5
    threshold_l = round(np.median(cent))*0.1

    less_noise = AudioEffectsChain().lowshelf(gain=-30.0, frequency=threshold_l, slope=0.8).highshelf(gain=-12.0, frequency=threshold_h, slope=0.5)#.limiter(gain=6.0)
    y_clean = less_noise(y)

    return y_clean


In [ ]:
test1Transformed_rnp = reduce_noise_power(test1Transformed, sr=44100)

In [ ]:
genWave(test1Transformed_rnp.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1Transformed_rnp.wav', sampleRate=44100, channels=1)

In [ ]:
'''------------------------------------
NOISE REDUCTION USING CENTROID ANALYSIS:
    receives an audio matrix,
    returns the matrix after gain reduction on noise
------------------------------------'''

def reduce_noise_centroid_s(y, sr):

    cent = librosa.feature.spectral_centroid(y=y, sr=sr)

    threshold_h = np.max(cent)
    threshold_l = np.min(cent)

    less_noise = AudioEffectsChain().lowshelf(gain=-12.0, frequency=threshold_l, slope=0.5).highshelf(gain=-12.0, frequency=threshold_h, slope=0.5).limiter(gain=6.0)

    y_cleaned = less_noise(y)

    return y_cleaned


In [ ]:
test1Transformed_rnc = reduce_noise_centroid_s(test1Transformed, sr=44100)

In [ ]:
genWave(test1Transformed_rnc.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1Transformed_rnc.wav', sampleRate=44100, channels=1)

In [ ]:
def reduce_noise_centroid_mb(y, sr):

    cent = librosa.feature.spectral_centroid(y=y, sr=sr)

    threshold_h = np.max(cent)
    threshold_l = np.min(cent)

    less_noise = AudioEffectsChain().lowshelf(gain=-30.0, frequency=threshold_l, slope=0.5).highshelf(gain=-30.0, frequency=threshold_h, slope=0.5).limiter(gain=10.0)
    # less_noise = AudioEffectsChain().lowpass(frequency=threshold_h).highpass(frequency=threshold_l)
    y_cleaned = less_noise(y)


    cent_cleaned = librosa.feature.spectral_centroid(y=y_cleaned, sr=sr)
    columns, rows = cent_cleaned.shape
    boost_h = math.floor(rows/3*2)
    boost_l = math.floor(rows/6)
    boost = math.floor(rows/3)

    # boost_bass = AudioEffectsChain().lowshelf(gain=20.0, frequency=boost, slope=0.8)
    boost_bass = AudioEffectsChain().lowshelf(gain=16.0, frequency=boost_h, slope=0.5)#.lowshelf(gain=-20.0, frequency=boost_l, slope=0.8)
    y_clean_boosted = boost_bass(y_cleaned)

    return y_clean_boosted

In [ ]:
test1Transformed_rncmb = reduce_noise_centroid_mb(test1Transformed, sr=44100)

In [ ]:
genWave(test1Transformed_rncmb.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1Transformed_rncmb.wav', sampleRate=44100, channels=1)

In [ ]:
'''------------------------------------
NOISE REDUCTION USING MFCC:
    receives an audio matrix,
    returns the matrix after gain reduction on noise
------------------------------------'''
def reduce_noise_mfcc_down(y, sr):

    hop_length = 512

    ## librosa
    # mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
    # librosa.mel_to_hz(mfcc)

    ## mfcc
    mfcc = python_speech_features.base.mfcc(y)
    mfcc = python_speech_features.base.logfbank(y)
    mfcc = python_speech_features.base.lifter(mfcc)

    sum_of_squares = []
    index = -1
    for r in mfcc:
        sum_of_squares.append(0)
        index = index + 1
        for n in r:
            sum_of_squares[index] = sum_of_squares[index] + n**2

    strongest_frame = sum_of_squares.index(max(sum_of_squares))
    hz = python_speech_features.base.mel2hz(mfcc[strongest_frame])

    max_hz = max(hz)
    min_hz = min(hz)

    speech_booster = AudioEffectsChain().highshelf(frequency=min_hz*(-1)*1.2, gain=-12.0, slope=0.6).limiter(gain=8.0)
    y_speach_boosted = speech_booster(y)

    return (y_speach_boosted)

In [ ]:
test1Transformed_rnmd = reduce_noise_mfcc_down(test1Transformed, sr=44100)

In [ ]:
genWave(test1Transformed_rnmd.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1Transformed_rnmd.wav', sampleRate=44100, channels=1)

In [ ]:
def reduce_noise_mfcc_up(y, sr):

    hop_length = 512

    ## librosa
    # mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
    # librosa.mel_to_hz(mfcc)

    ## mfcc
    mfcc = python_speech_features.base.mfcc(y)
    mfcc = python_speech_features.base.logfbank(y)
    mfcc = python_speech_features.base.lifter(mfcc)

    sum_of_squares = []
    index = -1
    for r in mfcc:
        sum_of_squares.append(0)
        index = index + 1
        for n in r:
            sum_of_squares[index] = sum_of_squares[index] + n**2

    strongest_frame = sum_of_squares.index(max(sum_of_squares))
    hz = python_speech_features.base.mel2hz(mfcc[strongest_frame])

    max_hz = max(hz)
    min_hz = min(hz)

    speech_booster = AudioEffectsChain().lowshelf(frequency=min_hz*(-1), gain=12.0, slope=0.5)#.highshelf(frequency=min_hz*(-1)*1.2, gain=-12.0, slope=0.5)#.limiter(gain=8.0)
    y_speach_boosted = speech_booster(y)

    return (y_speach_boosted)

In [ ]:
test1Transformed_rnmu = reduce_noise_mfcc_up(test1Transformed, sr=44100)

In [ ]:
genWave(test1Transformed_rnmu.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1Transformed_rnmu.wav', sampleRate=44100, channels=1)

In [ ]:
'''------------------------------------
NOISE REDUCTION USING MEDIAN:
    receives an audio matrix,
    returns the matrix after gain reduction on noise
------------------------------------'''

def reduce_noise_median(y, sr):
    y = sp.signal.medfilt(y,3)
    return (y)

In [ ]:
test1Transformed_rnmed = reduce_noise_median(test1Transformed, sr=44100)

In [ ]:
genWave(test1Transformed_rnmed.reshape(-1,1), '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/test1Transformed_rnmed.wav', sampleRate=44100, channels=1)

#### Limit signal length (to fit into the model input layer)

In [ ]:
fix_length = 110250
test1Transformed_res = librosa.util.fix_length(test1Transformed, size=fix_length)
test1Transformed_rnp_res = librosa.util.fix_length(test1Transformed_rnp, size=fix_length)
test1Transformed_rnc_res = librosa.util.fix_length(test1Transformed_rnc, size=fix_length)
test1Transformed_rncmb_res = librosa.util.fix_length(test1Transformed_rncmb, size=fix_length)
test1Transformed_rnmd_res = librosa.util.fix_length(test1Transformed_rnmd, size=fix_length)
test1Transformed_rnmu_res = librosa.util.fix_length(test1Transformed_rnmu, size=fix_length)
test1Transformed_rnmed_res = librosa.util.fix_length(test1Transformed_rnmed, size=fix_length)

#### Calculate MFCCs

In [ ]:
test1Transformed_mfcc = np.expand_dims(np.mean(librosa.feature.mfcc(test1Transformed_res, sr=44100, n_mfcc=13),
                                               axis=0).reshape(-1,1),
                                       axis=0)

In [ ]:
test1Transformed_rnp_mfcc = np.expand_dims(np.mean(librosa.feature.mfcc(test1Transformed_rnp_res, sr=44100, n_mfcc=13),
                                                   axis=0).reshape(-1,1),
                                           axis=0)

In [ ]:
test1Transformed_rnc_mfcc = np.expand_dims(np.mean(librosa.feature.mfcc(test1Transformed_rnc_res, sr=44100, n_mfcc=13),
                                                   axis=0).reshape(-1,1),
                                           axis=0)

In [ ]:
test1Transformed_rncmb_mfcc = np.expand_dims(np.mean(librosa.feature.mfcc(test1Transformed_rncmb_res, sr=44100, n_mfcc=13),
                                                     axis=0).reshape(-1,1),
                                             axis=0)

In [ ]:
test1Transformed_rnmd_mfcc = np.expand_dims(np.mean(librosa.feature.mfcc(test1Transformed_rnmd_res, sr=44100, n_mfcc=13),
                                                    axis=0).reshape(-1,1),
                                            axis=0)

In [ ]:
test1Transformed_rnmu_mfcc = np.expand_dims(np.mean(librosa.feature.mfcc(test1Transformed_rnmu_res, sr=44100, n_mfcc=13),
                                                    axis=0).reshape(-1,1),
                                            axis=0)

In [ ]:
test1Transformed_rnmed_mfcc = np.expand_dims(np.mean(librosa.feature.mfcc(test1Transformed_rnmed_res, sr=44100, n_mfcc=13),
                                                     axis=0).reshape(-1,1),
                                             axis=0)

#### Load Keras model

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model("Emotion_Voice_Detection_Model.h5")

##### Plain audio

In [ ]:
test1Transformed_pred = model.predict(test1Transformed_mfcc)

In [ ]:
test1Transformed_pred.argmax(axis=1)

In [ ]:
test1Transformed_pred

##### Noise Reduction Using Power

In [ ]:
test1Transformed_rnp_pred = model.predict(test1Transformed_rnp_mfcc)

In [ ]:
test1Transformed_rnp_pred.argmax(axis=1)

In [ ]:
test1Transformed_rnp_pred

##### Noise Reduction Using Centroid Analysis

In [ ]:
test1Transformed_rnc_pred = model.predict(test1Transformed_rnc_mfcc)

In [ ]:
test1Transformed_rnc_pred.argmax(axis=1)

In [ ]:
test1Transformed_rnc_pred

##### Noise Reduction Using Centroid Max Boost

In [ ]:
test1Transformed_rncmb_pred = model.predict(test1Transformed_rncmb_mfcc)

In [ ]:
test1Transformed_rncmb_pred.argmax(axis=1)

In [ ]:
test1Transformed_rncmb_pred

##### Noise Reduction MFCCs down

In [ ]:
test1Transformed_rnmd_pred = model.predict(test1Transformed_rnmd_mfcc)

In [ ]:
test1Transformed_rnmd_pred.argmax(axis=1)

In [ ]:
test1Transformed_rnmd_pred

##### Noise Reduction MFCCs up

In [ ]:
test1Transformed_rnmu_pred = model.predict(test1Transformed_rnmu_mfcc)

In [ ]:
test1Transformed_rnmu_pred.argmax(axis=1)

In [ ]:
test1Transformed_rnmu_pred

##### Noise Reduction Using Median

In [ ]:
test1Transformed_rnmed_pred = model.predict(test1Transformed_rnmed_mfcc)

In [ ]:
test1Transformed_rnmed_pred.argmax(axis=1)

In [ ]:
test1Transformed_rnmed_pred

#### Mixing voice and noise

In [38]:
from pydub import AudioSegment
from pydub.playback import play

In [ ]:
voice = AudioSegment.from_file('/home/datahack/Documents/RAVDESS/Audio_Speech_Actors_01-24/Actor_01/03-01-07-02-02-01-01.wav', format='wav') + 6

In [ ]:
noise = AudioSegment.from_file('/home/datahack/catkin_ws/src/mic2wav/src/wav_files/noise_mono.wav', format="wav")

In [ ]:
combined = voice.overlay(noise, loop=True)

In [ ]:
combined.export("/home/datahack/catkin_ws/src/mic2wav/src/wav_files/combined.wav", format='wav')

In [ ]:
play(combined)

### Remaking the dataset

In [67]:
import glob
import os
import random

In [86]:
VOICE_TYPE="Speech"
SPEECH_PATH_ORIG = "/home/datahack/Documents/RAVDESS/Audio_" + VOICE_TYPE + "_Actors_01-24"
SPEECH_PATH_DEST = "/home/datahack/Documents/RAVDESS_DEST/Audio_" + VOICE_TYPE + "_Actors_01-24"
NOISE_PATH = "/home/datahack/Documents/NOISE"

In [87]:
if not os.path.exists(SPEECH_PATH_DEST):
    print("CREATING " + SPEECH_PATH_DEST + "...")
    os.makedirs(SPEECH_PATH_DEST)
    print("DONE")
else:
    print("Directory already exists...")

CREATING /home/datahack/Documents/RAVDESS_DEST/Audio_Speech_Actors_01-24...
DONE


#### Load noise list

In [81]:
if os.path.exists(NOISE_PATH):
    noise_audios = glob.glob(os.path.join(NOISE_PATH,"noise*wav"))

In [82]:
random.choice(noise_audios)

'/home/datahack/Documents/NOISE/noise_08.wav'

#### Mixing audio and noise

In [83]:
def noisify(signal, segment, sig_volume=15, seg_volume=0, audio_type='wav'):
    voice = AudioSegment.from_file(signal, format=audio_type) + sig_volume
    noise = AudioSegment.from_file(segment, format=audio_type) + seg_volume
    
    print("Converting")
    return voice.overlay(noise, loop=True)

#### Reading input files...

In [88]:
speech_dirs_ori = glob.glob(SPEECH_PATH_ORIG + "/*")

In [89]:
count = {}
for sdir in speech_dirs_ori:
    basename = os.path.basename(sdir)
    speech_dir_dest = os.path.join(SPEECH_PATH_DEST, basename)
    speech_audios = glob.glob(os.path.join(SPEECH_PATH_ORIG,sdir,"*.wav"))
    for sfile in speech_audios:        
        #print("-------------------------------------")
        sfile_name = os.path.basename(sfile)
        #print(sfile_name)        
        sfile_name_desc = os.path.splitext(sfile_name)[0].split("-")
        # Modality: 01 Full Audio-Video 02 Video only 03 Audio only        
        modality = sfile_name_desc[0]
        
        # Vocal Channel: 01 Speech 02 Song
        vocal_channel = sfile_name_desc[1]
        
        # Emotion 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
        emotion_code = sfile_name_desc[2]
        if (emotion_code == '01'):
            emotion = 'neutral'
        elif (emotion_code == '02'):
            emotion = 'calm'
        elif (emotion_code == '03'):
            emotion = 'happy'
        elif (emotion_code == '04'):
            emotion = 'sad'
        elif (emotion_code == '05'):
            emotion = 'angry'
        elif (emotion_code == '06'):
            emotion = 'fearful'
        elif (emotion_code == '07'):
            emotion = 'disgust'
        elif (emotion_code == '08'):
            emotion = 'surprised'
        
        try:
            count[emotion] += 1
        except KeyError:
            count[emotion] = 0
        
        # Emotional intensity 01 = normal, 02 = strong. NOTE: There is no strong intensity for the ‘neutral’ emotion.
        emotion_intensity = sfile_name_desc[3]
        
        # Statement 01 = “Kids are talking by the door”, 02 = “Dogs are sitting by the door”
        statement = sfile_name_desc[4] 
        
        # Repetition 01 = 1st repetition, 02 = 2nd repetition
        repetition =  sfile_name_desc[5]
        
        # Actor 01 to 24. Odd numbered actors are male, even numbered actors are female
        actor = sfile_name_desc[6]

        file_name = VOICE_TYPE + "_" + emotion + '_' + str(count[emotion]).zfill(4) + '.wav'
        nfile = random.choice(noise_audios)
        print(nfile)
        noisy_speech = noisify(sfile, nfile)
        
        AudioSegment.export(noisy_speech, os.path.join(SPEECH_PATH_DEST,file_name), format='wav')

/home/datahack/Documents/NOISE/noise_24.wav
Converting
/home/datahack/Documents/NOISE/noise_16.wav
Converting
/home/datahack/Documents/NOISE/noise_11.wav
Converting
/home/datahack/Documents/NOISE/noise_16.wav
Converting
/home/datahack/Documents/NOISE/noise_21.wav
Converting
/home/datahack/Documents/NOISE/noise_19.wav
Converting
/home/datahack/Documents/NOISE/noise_23.wav
Converting
/home/datahack/Documents/NOISE/noise_17.wav
Converting
/home/datahack/Documents/NOISE/noise_03.wav
Converting
/home/datahack/Documents/NOISE/noise_17.wav
Converting
/home/datahack/Documents/NOISE/noise_06.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_16.wav
Converting
/home/datahack/Documents/NOISE/noise_14.wav
Converting
/home/datahack/Documents/NOISE/noise_03.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_14.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting
/home/data

Converting
/home/datahack/Documents/NOISE/noise_22.wav
Converting
/home/datahack/Documents/NOISE/noise_23.wav
Converting
/home/datahack/Documents/NOISE/noise_19.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_21.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_02.wav
Converting
/home/datahack/Documents/NOISE/noise_16.wav
Converting
/home/datahack/Documents/NOISE/noise_05.wav
Converting
/home/datahack/Documents/NOISE/noise_05.wav
Converting
/home/datahack/Documents/NOISE/noise_01.wav
Converting
/home/datahack/Documents/NOISE/noise_06.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_18.wav
Converting
/home/datahack/Documents/NOISE/noise_10.wav
Converting
/home/datahack/Documents/NOISE/noise_07.wav
Converting
/home/datahack/Documents/NOISE/noise_22.wav
Converting

Converting
/home/datahack/Documents/NOISE/noise_04.wav
Converting
/home/datahack/Documents/NOISE/noise_16.wav
Converting
/home/datahack/Documents/NOISE/noise_16.wav
Converting
/home/datahack/Documents/NOISE/noise_01.wav
Converting
/home/datahack/Documents/NOISE/noise_06.wav
Converting
/home/datahack/Documents/NOISE/noise_17.wav
Converting
/home/datahack/Documents/NOISE/noise_05.wav
Converting
/home/datahack/Documents/NOISE/noise_11.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting
/home/datahack/Documents/NOISE/noise_07.wav
Converting
/home/datahack/Documents/NOISE/noise_01.wav
Converting
/home/datahack/Documents/NOISE/noise_21.wav
Converting
/home/datahack/Documents/NOISE/noise_24.wav
Converting
/home/datahack/Documents/NOISE/noise_19.wav
Converting
/home/datahack/Documents/NOISE/noise_24.wav
Converting
/home/datahack/Documents/NOISE/noise_11.wav
Converting
/home/datahack/Documents/NOISE/noise_04.wav
Converting

/home/datahack/Documents/NOISE/noise_05.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_03.wav
Converting
/home/datahack/Documents/NOISE/noise_06.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_03.wav
Converting
/home/datahack/Documents/NOISE/noise_12.wav
Converting
/home/datahack/Documents/NOISE/noise_12.wav
Converting
/home/datahack/Documents/NOISE/noise_02.wav
Converting
/home/datahack/Documents/NOISE/noise_03.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_25.wav
Converting
/home/datahack/Documents/NOISE/noise_18.wav
Converting
/home/datahack/Documents/NOISE/noise_20.wav
Converting
/home/datahack/Documents/NOISE/noise_20.wav
Converting
/home/datahack/Documents/NOISE/noise_14.wav
Converting
/home/datahack/Documents/NOISE/noise_24.wav
Converting
/home/datahack/Documents/NOISE/noise_25.wav
Converting
/home/data

/home/datahack/Documents/NOISE/noise_07.wav
Converting
/home/datahack/Documents/NOISE/noise_04.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_23.wav
Converting
/home/datahack/Documents/NOISE/noise_04.wav
Converting
/home/datahack/Documents/NOISE/noise_19.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_13.wav
Converting
/home/datahack/Documents/NOISE/noise_10.wav
Converting
/home/datahack/Documents/NOISE/noise_06.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_23.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_10.wav
Converting
/home/datahack/Documents/NOISE/noise_18.wav
Converting
/home/datahack/Documents/NOISE/noise_04.wav
Converting
/home/datahack/Documents/NOISE/noise_13.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting
/home/data

Converting
/home/datahack/Documents/NOISE/noise_12.wav
Converting
/home/datahack/Documents/NOISE/noise_11.wav
Converting
/home/datahack/Documents/NOISE/noise_25.wav
Converting
/home/datahack/Documents/NOISE/noise_19.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting
/home/datahack/Documents/NOISE/noise_01.wav
Converting
/home/datahack/Documents/NOISE/noise_19.wav
Converting
/home/datahack/Documents/NOISE/noise_05.wav
Converting
/home/datahack/Documents/NOISE/noise_04.wav
Converting
/home/datahack/Documents/NOISE/noise_16.wav
Converting
/home/datahack/Documents/NOISE/noise_12.wav
Converting
/home/datahack/Documents/NOISE/noise_01.wav
Converting
/home/datahack/Documents/NOISE/noise_25.wav
Converting
/home/datahack/Documents/NOISE/noise_05.wav
Converting
/home/datahack/Documents/NOISE/noise_20.wav
Converting
/home/datahack/Documents/NOISE/noise_07.wav
Converting
/home/datahack/Documents/NOISE/noise_17.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting

Converting
/home/datahack/Documents/NOISE/noise_10.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_23.wav
Converting
/home/datahack/Documents/NOISE/noise_21.wav
Converting
/home/datahack/Documents/NOISE/noise_09.wav
Converting
/home/datahack/Documents/NOISE/noise_13.wav
Converting
/home/datahack/Documents/NOISE/noise_17.wav
Converting
/home/datahack/Documents/NOISE/noise_07.wav
Converting
/home/datahack/Documents/NOISE/noise_10.wav
Converting
/home/datahack/Documents/NOISE/noise_21.wav
Converting
/home/datahack/Documents/NOISE/noise_26.wav
Converting
/home/datahack/Documents/NOISE/noise_17.wav
Converting
/home/datahack/Documents/NOISE/noise_11.wav
Converting
/home/datahack/Documents/NOISE/noise_17.wav
Converting
/home/datahack/Documents/NOISE/noise_25.wav
Converting
/home/datahack/Documents/NOISE/noise_21.wav
Converting

Converting
/home/datahack/Documents/NOISE/noise_02.wav
Converting
/home/datahack/Documents/NOISE/noise_07.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting
/home/datahack/Documents/NOISE/noise_13.wav
Converting
/home/datahack/Documents/NOISE/noise_12.wav
Converting
/home/datahack/Documents/NOISE/noise_15.wav
Converting
/home/datahack/Documents/NOISE/noise_07.wav
Converting
/home/datahack/Documents/NOISE/noise_11.wav
Converting
/home/datahack/Documents/NOISE/noise_09.wav
Converting
/home/datahack/Documents/NOISE/noise_24.wav
Converting
/home/datahack/Documents/NOISE/noise_04.wav
Converting
/home/datahack/Documents/NOISE/noise_08.wav
Converting
/home/datahack/Documents/NOISE/noise_10.wav
Converting
/home/datahack/Documents/NOISE/noise_24.wav
Converting
/home/datahack/Documents/NOISE/noise_23.wav
Converting
/home/datahack/Documents/NOISE/noise_01.wav
Converting
/home/datahack/Documents/NOISE/noise_13.wav
Converting
/home/datahack/Documents/NOISE/noise_24.wav
Converting

In [ ]:
glob.glob("/home/datahack/Documents/RAVDESS/Audio_Speech_Actors_01-24/*")